In [2]:
pip install trdg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
Reason for being yanked: Doesn't work with Python 2
  Created wheel for diffimg: filename=diffimg-0.2.3-py3-none-any.whl size=4019 sha256=504fa5c5e1eb1bcb1d7e4a8349f805370e78089aa5b5f84f7f0cc67bc0d7f562
  Stored in directory: /root/.cache/pip/wheels/90/57/92/d4259a51f20cf92a473b567a009674e3390852b7fab19be6dc
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=88dad1e9e9ec6c7bece17a78fd054a824d80d44e04463ec6ad49dfd37286c02e
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built diffimg wikipedia


In [3]:
%cd "/content/drive/MyDrive/Colab Notebooks/OCR_proj/trdg"

/content/drive/MyDrive/Colab Notebooks/OCR_proj/trdg


In [4]:
from trdg.generators import GeneratorFromStrings
from PIL import Image
import os
import csv
import random
import string

# Define output directory and CSV file path
output_dir = "ocr_dataset4"
os.makedirs(output_dir, exist_ok=True)
csv_file = os.path.join(output_dir, "labels.csv")

# Function to generate random words
def generate_random_word(length=10):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

# Function to save images with labels
def save_handwritten_text_images(output_dir, csv_file, num_samples=10):
    # Generate random words
    random_words = [generate_random_word(10) for _ in range(num_samples)]

    # Create generator for handwritten text
    generator = GeneratorFromStrings(
        random_words,
        blur=0,  # No blur
        random_blur=False,
        distorsion_type=0,  # No distortion
        size=32,  # Font size
        language="en"  # Language set to English
    )

    labels = []
    fixed_width = 200
    fixed_height = 50

    for count, (img, lbl) in enumerate(generator):
        if count >= num_samples:
            break
        img = img.convert("L")  # Convert image to grayscale
        # Resize the image to fixed dimensions
        img = img.resize((fixed_width, fixed_height), Image.ANTIALIAS)
        # Save image with related filename
        img_filename = os.path.join(output_dir, f"{count+1}.png")
        img.save(img_filename)
        print(f"Saved {img_filename} with label {lbl}")
        labels.append(lbl)

    # Save labels to CSV
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Label"])
        for label in labels:
            writer.writerow([label])

# Generate and save images
save_handwritten_text_images(output_dir, csv_file, num_samples=100)

Saved ocr_dataset4/1.png with label fqqzkskjcj
Saved ocr_dataset4/2.png with label kmsgnanaxe
Saved ocr_dataset4/3.png with label rhlpxbaxbn
Saved ocr_dataset4/4.png with label nsywmwwoti
Saved ocr_dataset4/5.png with label wwpnpmlsmv
Saved ocr_dataset4/6.png with label yynpbssvlh
Saved ocr_dataset4/7.png with label nqnkdrywgt
Saved ocr_dataset4/8.png with label gjfqcaayir
Saved ocr_dataset4/9.png with label zzyomaotuq
Saved ocr_dataset4/10.png with label mgkykufhuo
Saved ocr_dataset4/11.png with label vhfsngrsvw
Saved ocr_dataset4/12.png with label ezwkvuxzsd
Saved ocr_dataset4/13.png with label blrizlughq
Saved ocr_dataset4/14.png with label qtnghdmdzl
Saved ocr_dataset4/15.png with label qpuapyajlv
Saved ocr_dataset4/16.png with label zoomdfqdqo
Saved ocr_dataset4/17.png with label wxnyhmlmkc
Saved ocr_dataset4/18.png with label taxwlqduna
Saved ocr_dataset4/19.png with label ugpmuztxhp
Saved ocr_dataset4/20.png with label ernjxzssbp
Saved ocr_dataset4/21.png with label wtkowzrvga
S